In [6]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Clustering/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.3. Sociodemographic Clustering/Output')
cur_date = "123122"

library(readxl)
library(tidyverse)
library(reshape2)
suppressPackageStartupMessages(library(dendextend))
library(pheatmap)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_121222.xlsx", sheet = 2))

# need to make some of the cols numeric
acs_df[,4:30] = apply(acs_df[,4:30], 2, as.numeric)
wildfire_hazard_df[,4:6] = apply(wildfire_hazard_df[,4:6], 2, as.numeric)

Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric

In [7]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,20.8,14.6,26.4,28.0,54.8,0,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,33.0,19.2,27.3,37.8,44.6,0,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,17.3,22.0,18.0,28.3,21.0,NA,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,26.8,14.0,30.0,33.3,19.9,NA,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,12.2,7.1,12.3,14.8,18.9,0,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,22.3,19.2,17.7,31.0,27.8,100,14.2,19.5,14.7,25.0


,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,278233.8,92.23703,1,0-20%
3,14000US37001020301,203.01,Alamance,278233.8,92.23703,1,0-20%
4,14000US37001020302,203.02,Alamance,278233.8,92.23703,1,0-20%
5,14000US37001020400,204.00,Alamance,278233.8,92.23703,1,0-20%
6,14000US37001020501,205.01,Alamance,278233.8,92.23703,1,0-20%


In [37]:
# combining into 1 df
acs_whp_df = inner_join(acs_df, wildfire_hazard_df) %>%
    # removing an irrelvant col
    select(-Avg_ACRES) %>%
    pivot_longer(cols = 4:30, names_to = "Variable", values_to = "Variable_Value") %>%
    # scaling within each variable
    group_by(Variable) %>%
    mutate(Norm_WHP = scale(Wildfire_Hazard_Potential_Mean)) %>%
    ungroup() %>%
    # taking avg for each county to make the analysis more simple
    group_by(County) %>%
    mutate(Norm_WHP = mean(Norm_WHP)) %>%
    ungroup() #%>%
    #pivot_wider(names_from = "Variable", values_from = "Variable_Value")

head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")


GEO_ID,Census_Tract,County,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage,Variable,Variable_Value,Norm_WHP
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
14000US37001020100,201,Alamance,92.23703,1,0-20%,Sex_Ratio,87.2,-0.617216
14000US37001020100,201,Alamance,92.23703,1,0-20%,Median_Age,37.3,-0.617216
14000US37001020100,201,Alamance,92.23703,1,0-20%,Population_Under_18,21.9,-0.617216
14000US37001020100,201,Alamance,92.23703,1,0-20%,Population_Over_18,78.1,-0.617216
14000US37001020100,201,Alamance,92.23703,1,0-20%,Population_Over_65,15.1,-0.617216
14000US37001020100,201,Alamance,92.23703,1,0-20%,Race_White,61.0,-0.617216


In [35]:
# # determining average variable concentrations to order the dataframe for the heat map
# avg_variable_df = acs_whp_df %>%
#     group_by(Variable) %>%
#     # dropping missing values for now
#     #drop_na() %>%
#     #summarize(Avg_Norm_Value = mean(Norm_WHP, na.rm = TRUE))%>%
#     #arrange(-Avg_Norm_Value)
#     arrange(-Norm_WHP)

# head(avg_variable_df)

# ordered_sociodemographic_variables = avg_variable_df$Variable

In [38]:
# creating a wider data for plotting that contains counties, sociodemographic variables, and WHP
clustering_df = acs_whp_df[,c(3,7,9)] #%>%
    #pivot_wider(names_from = Variable, values_from = Norm_WHP) %>%
    #column_to_rownames(var = "County")

# ordering cytokines for heat map
# clustering_df = clustering_df[ordered_sociodemographic_variables,]

head(clustering_df)

County,Variable,Norm_WHP
<chr>,<chr>,<dbl>
Alamance,Sex_Ratio,-0.617216
Alamance,Median_Age,-0.617216
Alamance,Population_Under_18,-0.617216
Alamance,Population_Over_18,-0.617216
Alamance,Population_Over_65,-0.617216
Alamance,Race_White,-0.617216
